In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# openPMD beamphysics examples

In [2]:
from opmd_beamphysics import ParticleGroup
from opmd_beamphysics.particles import particle_paths, all_components, component_str

from h5py import File



In [3]:
# Open a file, fine the particle paths from the root attributes
H5FILE = 'data/bmad_particles.h5'
h5 = File(H5FILE)


ppaths = particle_paths(h5)
ppaths

['/data/00001/particles/']

In [4]:
# Point to particles
ph5 = h5[ppaths[0]]
list(ph5)

['branchIndex',
 'elementIndex',
 'locationInElement',
 'momentum',
 'particleStatus',
 'position',
 'sPosition',
 'spin',
 'time',
 'timeOffset',
 'totalMomentum',
 'totalMomentumOffset',
 'weight']

In [5]:
# Search for all valid components
all_components(ph5)

['branchIndex',
 'elementIndex',
 'locationInElement',
 'momentum/x',
 'momentum/y',
 'momentum/z',
 'particleStatus',
 'position/x',
 'position/y',
 'position/z',
 'sPosition',
 'spin/x',
 'spin/y',
 'spin/z',
 'time',
 'timeOffset',
 'totalMomentum',
 'totalMomentumOffset',
 'weight']

In [6]:
# Get some info
for component in all_components(ph5):
    info = component_str(ph5, component)
    print(info)

branchIndex [constant [0] with shape [10000]]
elementIndex [constant [8] with shape [10000]]
locationInElement [constant [1] with shape [10000]]
momentum/x [10000 items] is a momentum with units: kg*m/s
momentum/y [10000 items] is a momentum with units: kg*m/s
momentum/z [10000 items] is a momentum with units: kg*m/s
particleStatus [constant [1] with shape [10000]]
position/x [10000 items] is a length with units: m
position/y [10000 items] is a length with units: m
position/z [constant [0.] with shape [10000]] is a length with units: m
sPosition [constant [0.445] with shape [10000]] is a length with units: m
spin/x [constant [0.] with shape [10000]]
spin/y [constant [0.] with shape [10000]]
spin/z [constant [0.] with shape [10000]]
time [10000 items] is a time with units: s
timeOffset [10000 items] is a time with units: s
totalMomentum [10000 items] is a momentum with units: kg*m/s
totalMomentumOffset [constant [2.24443398e-20] with shape [10000]] is a momentum with units: kg*m/s
weigh

# ParticleGroup class

In [7]:
# Parse all these into a class
P = ParticleGroup(ph5)

In [8]:
?P

Type:        ParticleGroup
String form: <opmd_beamphysics.particles.ParticleGroup object at 0x118fd8b10>
File:        ~/Code/GitHub/openPMD-beamphysics/opmd_beamphysics/particles.py
Docstring:  
Particle Group class

Initialized on on openPMD beamphysics particle group.

The fundamental bunch data is stored in __dict__ with keys
    str: species
    int: n_particle
    np.array: x, px, y, py, z, pz, t, status, weight
where:
    x, y, z are positions in units of [m]
    px, py, pz are momenta in units of [eV/c]
    t is time in [s]
    weight is the macro-charge weight in C, used for statistical calulations.
    
Derived data can be computed as properties:
    gamma, beta, beta_x, beta_y, beta_z: relativistic factors
    energy: energy in eV
    p: total momentum in eV
    mass: rest mass in eV
    
Particles are often stored at the same time (i.e. from a t-based code), 
or with the same z position (i.e. from an s-based code.)
Routines: drift_to_z and drift_to_t help to convert these.
 

In [9]:
# x positions, in meters
P.x

array([-7.84703367e-05,  4.65304310e-05, -9.52450043e-05, ...,
        4.59195263e-05,  4.03434287e-05,  3.09875028e-05])

In [10]:
# relativistic gamma, calculated on the fly
P.gamma

array([82.19188175, 82.19279119, 82.19020009, ..., 82.19079634,
       82.19137401, 82.19318991])

In [11]:
# Statistics on any of these
P.avg('gamma'), P.std('p')

(82.19149709866579, 600.0277476673164)

In [12]:
# Covariance matrix of any list of keys
P.cov('x', 'px', 'y', 'py')

array([[ 3.66679176e-09,  1.09983845e+00,  4.00403091e-15,
         2.80408658e-06],
       [ 1.09983845e+00,  4.01102270e+08,  1.25551199e-06,
         1.05247374e+03],
       [ 4.00403091e-15,  1.25551199e-06,  4.96199453e-09,
        -9.93277411e-01],
       [ 2.80408658e-06,  1.05247374e+03, -9.93277411e-01,
         2.51458057e+08]])

In [13]:
# Units
P.units('x'), P.units('energy')

('m', 'eV')

In [14]:
# These particles are from Bmad, at the same z and different times
P.std('z'), P.std('t')

(0.0, 3.000495719904134e-12)

In [15]:
# Get the central time
t0 = P.avg('t')
t0

1.484470349840882e-09

In [16]:
# Drift all particles to this time
P.drift_to_t(t0)

In [17]:
# Now these are at different z, and the same t
P.std('z'), P.avg('t'), set(P.t)

(0.0008994592399391498, 1.4844703498408813e-09, {1.484470349840882e-09})